In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import warnings
warnings.filterwarnings("ignore")

In [2]:
sku = pd.read_csv("skuinfo2.csv")

In [3]:
skuinfo = pd.read_csv("skuinfo2.csv")
skuinfo.shape

(1556033, 10)

In [4]:
skuinfo.head()

,SKU,DEPT,CLASSID,UPC,STYLE,COLOR,SIZE,PACKSIZE,VENDOR,BRAND
0,3,6505,113,400000003000,00 F55KT2,WHISPERWHITE,P8EA,1,5119207,TURNBURY
1,4,8101,002,400000004000,22 615CZ4,SPEARMI,S,1,3311144,C A SPOR
2,5,7307,003,400000005000,7LBS 245-01,34 SILVER,KING,1,5510554,BEAU IDE
3,8,3404,00B,400000008000,622 F05H84,MORNING MI,2T,1,2912827,HARTSTRI
4,15,2301,004,400000015000,126 MDU461,255CAMEL,12,1,23272,JONES/LA


In [5]:
skstinfo = pd.read_csv("skstinfo.csv", header = None, usecols = [0, 2, 3])

In [6]:
skstinfo.shape

(39230146, 3)

In [7]:
skstinfo.rename(columns = {0: "SKU",
                          2: "COST",
                          3: "RETAIL"}, inplace = True)

In [8]:
trnsact = pd.read_csv("trnsact.csv", header = None, usecols = [0, 3, 5, 6, 7, 9])

In [9]:
trnsact.rename(columns = {0: "SKU", 3: "TRANNUM", 5: "SALEDATE", 6: "STYPE", 7: "QUANTITY", 9: "AMT"}, inplace = True)

### *Note: Since we are collecting a 10% sample from each of the necessary dataframes, each time you run the below cell, you will get a different dataset*

In [10]:
# Select random 10% subsamples from original tables because the original tables are too large to perform joins
f_transact = trnsact.sample(round(len(trnsact) * 0.10))
f_sku = sku.sample(round(len(sku) * 0.10))
f_skst = skstinfo.sample(round(len(skstinfo) * 0.10))

In [11]:
# Join the three subsamples on SKU
merged_dfs = pd.merge(f_sku, f_skst, on = "SKU", how = "inner")

In [12]:
merged_dfs.shape

(388439, 12)

In [13]:
f_merged = pd.merge(merged_dfs, f_transact, on ="SKU", how = "inner")

In [14]:
f_merged.shape

(14783421, 17)

In [15]:
final_data = f_merged[["SKU", "VENDOR", "COST", "RETAIL", "TRANNUM", "SALEDATE", "STYPE", "QUANTITY", "AMT"]]

In [16]:
final_data.shape

(14783421, 9)

In [17]:
final_data.head()

,SKU,VENDOR,COST,RETAIL,TRANNUM,SALEDATE,STYPE,QUANTITY,AMT
0,9550334,3814938,84.50,199.00,1600,2005-08-24,P,1,199.00
1,9968561,2819205,10.88,17.00,500,2005-04-15,P,1,34.00
2,9968561,2819205,10.88,17.00,600,2005-06-14,P,1,25.50
3,9968561,2819205,10.88,17.00,900,2005-05-18,P,1,34.00
4,9968561,2819205,10.88,17.00,1300,2005-06-10,R,1,25.50


In [18]:
# Calculate average cost per vendor
final_data["total_cost"] = final_data["COST"] * final_data["QUANTITY"]
sub = final_data[["SKU", "VENDOR", "total_cost"]]
sub = sub.groupby("VENDOR").agg(CNT = ("SKU", "size"), TOTAL_COST = ("total_cost", "sum"))
sub["AVG_COST"] = sub["TOTAL_COST"] / sub["CNT"]
sub = sub.drop(columns=["CNT", "TOTAL_COST"])

final_data = final_data.drop(columns=["COST", "TRANNUM"])
final_data = final_data.merge(sub, how="left", on="VENDOR")

In [19]:
final_data

,SKU,VENDOR,RETAIL,SALEDATE,STYPE,QUANTITY,AMT,total_cost,AVG_COST
0,9550334,3814938,199.00,2005-08-24,P,1,199.00,84.50,55.50
1,9968561,2819205,17.00,2005-04-15,P,1,34.00,10.88,10.88
2,9968561,2819205,17.00,2005-06-14,P,1,25.50,10.88,10.88
3,9968561,2819205,17.00,2005-05-18,P,1,34.00,10.88,10.88
4,9968561,2819205,17.00,2005-06-10,R,1,25.50,10.88,10.88
...,...,...,...,...,...,...,...,...,...
14783416,5369250,13109,16.00,2005-08-20,P,1,16.00,5.75,5.41
14783417,5369250,13109,16.00,2005-07-31,R,1,16.00,5.75,5.41
14783418,5369250,13109,16.00,2005-07-18,P,1,16.00,5.75,5.41
14783419,5369250,13109,16.00,2005-08-20,P,1,16.00,5.75,5.41


In [20]:
def categorize(row, row_val):
    if row["STYPE"] == row_val:
        return 1
    else:
        return None
final_data['Purchases'] = final_data.apply(lambda row: categorize(row, "P"), axis=1)
final_data["Returns"] = final_data.apply(lambda row: categorize(row, "R"), axis = 1)

In [21]:
# Calculate recency of purchase
sub2 = final_data[final_data["STYPE"] == "P"]
sub2 = sub2.groupby("VENDOR").agg(RECENCY = ("SALEDATE", "max"), FREQUENCY = ("SKU", "size"))

final_data = final_data.merge(sub2, how="left", on="VENDOR")

In [22]:
final_data.head()

,SKU,VENDOR,RETAIL,SALEDATE,STYPE,QUANTITY,AMT,total_cost,AVG_COST,Purchases,Returns,RECENCY,FREQUENCY
0,9550334,3814938,199.00,2005-08-24,P,1,199.00,84.50,55.50,1.00,NaN,2005-08-27,2729.00
1,9968561,2819205,17.00,2005-04-15,P,1,34.00,10.88,10.88,1.00,NaN,2005-08-27,1032.00
2,9968561,2819205,17.00,2005-06-14,P,1,25.50,10.88,10.88,1.00,NaN,2005-08-27,1032.00
3,9968561,2819205,17.00,2005-05-18,P,1,34.00,10.88,10.88,1.00,NaN,2005-08-27,1032.00
4,9968561,2819205,17.00,2005-06-10,R,1,25.50,10.88,10.88,NaN,1.00,2005-08-27,1032.00


In [23]:
# Caluclate the profit for each SKU based on the sale type
final_data.loc[final_data.STYPE == "P", "PROFIT"] = (final_data["AMT"] - final_data["AVG_COST"]) * final_data["QUANTITY"]
final_data.loc[final_data.STYPE == "R", "PROFIT"] = (-final_data["AMT"] - final_data["AVG_COST"]) * final_data["QUANTITY"]

In [24]:
final_data.head(2)

,SKU,VENDOR,RETAIL,SALEDATE,STYPE,QUANTITY,AMT,total_cost,AVG_COST,Purchases,Returns,RECENCY,FREQUENCY,PROFIT
0,9550334,3814938,199.00,2005-08-24,P,1,199.00,84.50,55.50,1.00,NaN,2005-08-27,2729.00,143.50
1,9968561,2819205,17.00,2005-04-15,P,1,34.00,10.88,10.88,1.00,NaN,2005-08-27,1032.00,23.12


In [25]:
# Aggregate rows with the same VENDOR
f_data = final_data.groupby("VENDOR").agg(FREQUENCY = ("FREQUENCY", "max"),
                                          RECENCY = ("RECENCY", "max"),
                                          AVG_COST = ("AVG_COST", "mean"),
                                          TOTAL_COST = ("total_cost", "sum"),
                                          RETAIL = ("RETAIL", "sum"),
                                          QUANTITY = ("QUANTITY", "sum"),
                                          AMT = ("AMT", "sum"),
                                          PROFIT = ("PROFIT", "sum"),
                                          PURCHASES = ("Purchases", "sum"),
                                          RETURNS = ("Returns", "sum")).sort_values("FREQUENCY", ascending = False)

In [26]:
f_data = f_data.reset_index()
f_data.head(2)

,VENDOR,FREQUENCY,RECENCY,AVG_COST,TOTAL_COST,RETAIL,QUANTITY,AMT,PROFIT,PURCHASES,RETURNS
0,5511283,2125692.00,2005-08-27,9.84,21714578.10,36194305.00,2207718,35649370.87,10964289.67,2125692.00,82026.00
1,113645,887786.00,2005-08-27,15.88,14799962.40,24594196.52,931722,24283726.40,6826128.40,887786.00,43936.00


In [27]:
# Drop vendors with only records of return
f_data = f_data.dropna(subset=["FREQUENCY", "RECENCY"])

In [28]:
# Drop vendors with only records of return
f_data = f_data.dropna(subset=["FREQUENCY", "RECENCY"])

In [29]:
# Uncomment if you u wish to save data to ur local files

#f_data.to_csv("group7_data.csv")

### ROI Analysis
### *Note: Calculations done based on subset of total data, and percentages will change each time this notebook is run, but the should stay within similar percentage rates*

Based on the subset of data we selected below are the revenue, costs, and profits amounts

In [30]:
best_vendor_lst = [5511283, 113645, 13031, 5715232, 3626213,
                   2012863, 3313116, 9520439, 313319, 6016957, 
                   7016341, 2067178, 4516339, 13396, 1114936, 
                   2017178, 5011295, 60904, 10903, 6041161, 
                   4412768, 6013105, 9514659, 1116343, 6062767, 
                   66561, 226176, 4259203, 9113491, 7326340, 
                   5745232, 11679, 3916215, 514761, 4016074, 
                   5016699, 16561, 7510902]

worst_vendor_lst = [115362, 7045883, 7211456, 7095883, 7055883, 
                    7035883, 2819403, 6935883, 1012863, 16605, 
                    9716181, 2512827, 3513036, 5611454, 7221456, 
                    6945292, 9713322, 4913317, 2219404, 6916222]


f_data["VENDOR_CATEGORY"] = "Average Performing Vendors"
f_data.loc[f_data["VENDOR"].isin(worst_vendor_lst), "VENDOR_CATEGORY"] = "Worst Performing Vendors"
f_data.loc[f_data["VENDOR"].isin(best_vendor_lst), "VENDOR_CATEGORY"] = "Best Performing Vendors"

#keep the useful features 
f_data = f_data[["VENDOR","TOTAL_COST", "PROFIT", "VENDOR_CATEGORY"]]

f_data.head()

,VENDOR,TOTAL_COST,PROFIT,VENDOR_CATEGORY
0,5511283,21714578.10,10964289.67,Best Performing Vendors
1,113645,14799962.40,6826128.40,Best Performing Vendors
2,3626213,781101.23,707233.45,Best Performing Vendors
3,2012863,1154713.14,1147666.22,Best Performing Vendors
4,5715232,13331432.26,648184.71,Best Performing Vendors


- Total revenue, costs, and profits of all vendors
- According to https://www.ibisworld.com/industry-statistics/market-size/department-stores-united-states/#:~:text=What%20is%20the%20market%20size,is%20%24135.9bn%20in%202022. the US revenue for the departement store industry is $117.00 billion

In [31]:
df = pd.read_csv("group7_data.csv")
df[["VENDOR_CATEGORY", "TOTAL_COST"]] = f_data[["VENDOR_CATEGORY", "TOTAL_COST"]]
df = df[["VENDOR", "PROFIT", "TOTAL_COST", "VENDOR_CATEGORY" ]]
df.head()

,VENDOR,PROFIT,TOTAL_COST,VENDOR_CATEGORY
0,5511283,16926414.71,21714578.10,Best Performing Vendors
1,113645,13265587.62,14799962.40,Best Performing Vendors
2,13031,810849.46,781101.23,Best Performing Vendors
3,5715232,1292774.47,1154713.14,Best Performing Vendors
4,3626213,529786.74,13331432.26,Best Performing Vendors


In [32]:
revenue = round((df["TOTAL_COST"].sum()) + (df["PROFIT"].sum()), 0)
cost = round((df["TOTAL_COST"].sum()), 0)

profit = revenue - cost

print("revenue:",revenue)
print("cost:", cost)
print("profit:", profit)

print("-------------------")
global_rev = 117000000000
dill_perc_rev = round((revenue/global_rev * 100), 2)
print("Based on our subset of data Dillards revenue makes up {}% of of total revenue in the department store market in the United States".format(dill_perc_rev))

revenue: 288104711.0
cost: 194638007.0
profit: 93466704.0
-------------------
Based on our subset of data Dillards revenue makes up 0.25% of of total revenue in the department store market in the United States


Based on the vendors we labeled as "Best" below are the revenue, costs, and profit amounts

In [33]:
best_v = df[df["VENDOR_CATEGORY"] == "Best Performing Vendors"]
bv_rev = round(sum(best_v["TOTAL_COST"]) + sum(best_v["PROFIT"]), 0)
bv_cost = round(sum(best_v["TOTAL_COST"]), 0)
bv_prof = bv_rev - bv_cost

print("Best Vendors Revenue:", bv_rev)
print("Best Vendors Cost:", bv_cost)
print("Best_Vendors Profit:", bv_prof)

print("-------------------------------")
print("The best vendors make up {}% of total revenue".format(round((bv_rev/revenue)*100)))
print("The best vendors make up {}% of total cost".format(round((bv_cost/cost) * 100)))
print("The best vendors make up {}% of total profit".format(round((bv_prof/profit) * 100)))


Best Vendors Revenue: 154209118.0
Best Vendors Cost: 99480088.0
Best_Vendors Profit: 54729030.0
-------------------------------
The best vendors make up 54% of total revenue
The best vendors make up 51% of total cost
The best vendors make up 59% of total profit


Based on the vendors we labeled as "Worst" below are the revenue, costs, and profit amounts

In [34]:
worst_v = df[df["VENDOR_CATEGORY"] == "Worst Performing Vendors"]
wv_rev = round(sum(worst_v["TOTAL_COST"]) + sum(worst_v["PROFIT"]), 0)
wv_cost = round(sum(worst_v["TOTAL_COST"]), 0)
wv_prof = wv_rev - wv_cost

print("Worst Vendors Revenue:", wv_rev)
print("Worst Vendors Cost:", wv_cost)
print("Worst Vendors Profit:", wv_prof)

print("-------------------------------")
print("The worst vendors make up {}% of total revenue".format(round((wv_rev/revenue)*100)))
print("The worst vendors make up {}% of total cost".format(round((wv_cost/cost) * 100)))
print("The worst vendors make up {}% of total profit".format(round((wv_prof/profit) * 100)))


Worst Vendors Revenue: 4621982.0
Worst Vendors Cost: 4488822.0
Worst Vendors Profit: 133160.0
-------------------------------
The worst vendors make up 2% of total revenue
The worst vendors make up 2% of total cost
The worst vendors make up 0% of total profit


- Fte: 4 of us
- ds_salary: pulled from given excel sheet
- duration: we worked on this for about 12 weeks
- hrs: 4 * 40

In [35]:
fte = 4
ds_hrly_rt = 40
hrs = 4 *10

duration = 12
cloud_comp_hours = 8760
c_comp_cost = 0.10

consulting_cost = (duration * hrs) * ds_hrly_rt
cloud_computing_cost = cloud_comp_hours * c_comp_cost
print("The cost of our consulting is ${}".format(consulting_cost + cloud_computing_cost))

print("The ROI for Dillards using our analysis is: ${}".format(wv_cost - (consulting_cost + cloud_computing_cost)))

The cost of our consulting is $20076.0
The ROI for Dillards using our analysis is: $4468746.0


- By dropping the vendors labeled as worst Dillards can cut costs by $4 million or 2% of total cost in the next fiscal year

- By dropping the worst performing vendors, revenue and profit will initially decrease by 2%, however according to https://www.eiu.com/n/campaigns/consumer-in-2023/ we can expect the department store industry to grow by 6% we can expect to make up this loss in revenue in the next fiscal year

- We would reccomend Dillards to re-aclocate saved costs into investing more in the vendors we labeled as best in order to achieve higher growth in the next fiscal year

- The total ROI for our analysis is roughly $4 million